In [1]:
import json

with open("trees.geojson","r") as f:
 gjs = json.load(f)

In [2]:
print(len(gjs["features"]))

429292


In [3]:
from collections import Counter
trees = gjs["features"]
genus = Counter([t["properties"]["gattung"] for t in trees])
print(genus)

Counter({'TILIA': 150678, 'ACER': 87465, 'QUERCUS': 38171, 'PLATANUS': 24771, 'AESCULUS': 20284, 'FRAXINUS': 12008, 'BETULA': 11050, 'ROBINIA': 10167, 'CORYLUS': 8522, 'CARPINUS': 7883, 'PRUNUS': 7630, 'ULMUS': 7202, 'POPULUS': 6790, 'CRATAEGUS': 5621, 'SORBUS': 5019, 'PINUS': 3825, 'GLEDITSIA': 2957, 'PYRUS': 2404, 'SOPHORA': 1926, 'LIQUIDAMBAR': 1786, 'MALUS': 1359, 'GINKGO': 1292, 'SALIX': 1193, 'ALNUS': 1100, 'PICEA': 893, 'FAGUS': 880, 'AILANTHUS': 857, 'JUGLANS': 828, 'OSTRYA': 690, 'MAGNOLIA': 494, 'AMELANCHIER': 305, 'CATALPA': 263, 'LARIX': 254, 'PSEUDOTSUGA': 190, 'RHUS': 186, 'ZELKOVA': 182, 'MORUS': 181, 'CASTANEA': 175, 'PARROTIA': 169, 'TAXUS': 152, 'Abies': 147, 'CELTIS': 130, 'CORNUS': 125, 'UNBEKANNT': 107, 'LIRIODENDRON': 97, 'CHAMAECYPARIS': 82, 'THUJA': 79, 'KOELREUTERIA': 75, 'CERCIS': 71, 'ELAEAGNUS': 69, 'SAMBUCUS': 67, '': 42, 'TETRADIUM': 42, 'CERCIDIPHYLLUM': 39, 'CUPRESSOCYPARIS': 33, 'JUNIPERUS': 28, 'GYMNOCLADUS': 24, 'PHELLODENDRON': 23, 'PAULOWNIA': 23, '

In [7]:
import sys
!{sys.executable} -m pip install pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 36.8 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip


In [8]:
from pyproj import CRS, Transformer

# Define EPSG codes
epsg_25833 = CRS.from_epsg(25833)  # ETRS89 / UTM zone 33N
wgs84 = CRS.from_epsg(4326)  # WGS84

# Create transformer object
transformer = Transformer.from_crs(epsg_25833, wgs84)


def to_wgs(coordinates):
x_utm, y_utm = 378726.33999999985, 5823282.1251

# Perform transformation
    lon, lat = transformer.transform(coordinates[0], coordinates[1])

# Print the WGS84 coordinates
print(f"Longitude (WGS84): {lon:.6f}")
print(f"Latitude (WGS84): {lat:.6f}")

Longitude (WGS84): 52.546096
Latitude (WGS84): 13.211545


In [28]:
import csv

keys = ["id", "gattung", "art_dtsch", "lon", "lat"]

with open("trees.csv", "w", newline="") as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=keys)
    writer.writeheader()

    # Write each dictionary as a row
    for tree in trees:
        prop = tree["properties"]
        if prop["gattung"] not in ("MAGNOLIA", "PRUNUS", "PYRUS", "MALUS", "CRATAEGUS"):
            continue
        coordinates = tree["geometry"]["coordinates"]
        lat, lon = transformer.transform(coordinates[0], coordinates[1])
        writer.writerow({"id": prop["baumid"], "gattung": prop["gattung"], "art_dtsch": prop["art_dtsch"], "lon":lon, "lat":lat})

In [30]:
from collections import Counter
trees = gjs["features"]
genus = Counter([t["properties"]["gattung"] for t in trees if 'pflaume' in t["properties"]["art_dtsch"].lower()])
print(genus)

Counter({'PRUNUS': 750, 'CRATAEGUS': 162})


In [45]:
def filter_file(file_name, filter_func):
    new_features = []
    for tree in trees:
        prop = tree["properties"]
        if not filter_func(prop):
            continue
        coordinates = tree["geometry"]["coordinates"]
        lat, lon = transformer.transform(coordinates[0], coordinates[1])
        new_features.append({"type": "Feature", "geometry": {"type": "Point", "coordinates": [lon, lat]}, "properties":{ "art_dtsch": prop["art_dtsch"]}})
    
    gjs = {"type":"FeatureCollection", "features":new_features}
    
    with open(file_name,"w") as f:
        json.dump(gjs, f)


filter_file("magnolia.geojson", lambda t: t["gattung"] == "MAGNOLIA")
filter_file("cherry.geojson", lambda t: 'kirsche' in t["art_dtsch"].lower() and t["gattung"] != "CRATAEGUS")
filter_file("apple.geojson", lambda t: 'apfel' in t["art_dtsch"].lower() and t["gattung"] != "CRATAEGUS")
filter_file("pear.geojson", lambda t: 'birne' in t["art_dtsch"].lower() and t["gattung"] != "CRATAEGUS")
filter_file("plum.geojson", lambda t: 'pflaume' in t["art_dtsch"].lower() and t["gattung"] != "CRATAEGUS")